In [1]:
from dotenv import load_dotenv
from google import genai

load_dotenv()
gemini_client = genai.Client()

In [2]:
def list_available_embedding_models():
    for model in gemini_client.models.list():
        if model.supported_actions is None:
            continue
        if "embedContent" in model.supported_actions:
            print(model.name)

list_available_embedding_models()

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [3]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        pass

    def __call__(self, input: Documents) -> Embeddings:
        EMBEDDING_MODEL = "gemini-embedding-001"
        response = gemini_client.models.embed_content(
            model=EMBEDDING_MODEL,
            contents=input,
            config=types.EmbedContentConfig(
                title="Custom embedding",
                task_type="RETRIEVAL_DOCUMENT",
                output_dimensionality=3072,
            ),
        )
        if response.embeddings is None:
            raise RuntimeError("No embeddings returned")
        if response.embeddings[0].values is None:
            raise RuntimeError("No embedding values returned")
        return response.embeddings[0].values  # type: ignore

gemini_embedding_function = GeminiEmbeddingFunction()

In [4]:
import chromadb
from chromadb.errors import NotFoundError

chroma_client = chromadb.PersistentClient()
try:
    db = chroma_client.get_collection(
        name="rag-db", embedding_function=GeminiEmbeddingFunction()
    )
except NotFoundError:
    db = chroma_client.create_collection(
        name="rag-db", embedding_function=GeminiEmbeddingFunction()
    )

In [5]:
def embed_document(text, db):
    parts = []
    current_part = ""
    for c in text:
        current_part += c
        if len(current_part) >= 500 or c == "\n":
            current_part = current_part.strip()
            if current_part == "" or current_part.startswith("#"):
                current_part = ""
                continue
            parts.append(current_part.strip())
            current_part = ""

    for i, part in enumerate(parts):
        db.add(documents=[part], ids=[f"doc-{i}"])

with open("document.md", "r", encoding="utf-8") as f:
    document = f.read()

# embed_document(document, db)

In [35]:
def get_relevant_passages(query, db):
    passage = db.query(query_texts=[query], n_results=5)["documents"][0]
    return passage

In [7]:
def make_prompt(query, passage):
    prompt="""
        You are a helpful and informative bot that answers questions using text from the reference passage included below.
        Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
        If the passage is irrelevant to the answer, you may ignore it.

        QUESTION: '{query}'

        PASSAGE: '{passages}'
        
        ANSWER:
    """
    return prompt.format(
        query=query,
        passages=passage
    )

In [41]:
def generate_answer(query, passage):
    CHAT_MODEL = "gemini-2.5-flash-lite"
    response = gemini_client.models.generate_content(
        model=CHAT_MODEL,
        contents=make_prompt(query, passage),
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0),
        )
    )
    return (
        response.usage_metadata.total_token_count if response.usage_metadata else None,
        response.text,
    )

In [ ]:
def test_with_rag(query, db):
    passages = get_relevant_passages(query, db)
    answer = generate_answer(query, "".join(passages))
    return answer

def test_with_context(query, long_context):
    answer = generate_answer(query, long_context)
    return answer

## Testcases

In [42]:
query = "神經織網技術的發明者是誰？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (499, 'Reed 博士在 2011 年發明了「神經織網」（Neuro-Loom）系統。')
Context: (2608, '神經織網技術的發明者是天才工程師 Evelyn Reed 博士。')


In [43]:
query = "旨在規範織夢技術、於 2042 年頒布的法案叫什麼名字？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (641, '於 2042 年頒布、旨在規範腦機介面與意識干預技術的法案，即《關於規範腦機介面與意識干預技術的倫理準則》，被稱為《墨菲斯法案》（The "Morpheus" Act）。')
Context: (2634, '旨在規範織夢技術、於 2042 年頒布的法案稱為《墨菲斯法案》（The "Morpheus" Act）。')


In [44]:
query = "Aris Thorne 博士最初發起的專案計畫名稱是什麼？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (831, 'Aris Thorne 博士最初發起的專案計畫名稱是「Lucid」。')
Context: (2615, 'Aris Thorne 博士於 1985 年發起的專案計畫名稱是「Lucid」。')


In [45]:
query = "織夢技術的核心技術原理包含哪三個主要部分？請簡要說明它們各自的作用。"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (528, '織夢技術的實現依賴於三大核心技術的整合，其中「睡夢波誘導 (Somno-Wave Induction, SWI)」是織夢過程的第一步，該技術通過釋放特定頻率的聲光信號，引導使用者大腦進入適合織夢的非快速動眼期（NREM）第三階段和快速動眼期（REM）之間的過渡狀態，此時大腦記憶整理活躍且神經可塑性達峰值，便於外部引導和記憶存取。')
Context: (2715, '織夢技術的核心技術原理包含三個主要部分：睡夢波誘導（Somno-Wave Induction, SWI），它引導使用者大腦進入適合織夢的過渡狀態；記憶印痕存取（Memory Engram Access, MEA），它定位並讀取與特定記憶相關的「印痕細胞」集群，並能將新的資訊模式寫入；以及感官回饋模擬（Sensory Feedback Simulation, SFS），它通過電磁信號刺激大腦的感覺皮層，直接生成逼真的五感體驗。')


In [46]:
query = "為什麼織夢技術能夠應用於創傷後壓力症候群（PTSD）的治療？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (717, '織夢技術能夠應用於創傷後壓力症候群（PTSD）的治療，是因為它能夠在絕對安全的夢境環境中，引導患者以可控的方式反覆面對和處理他們的創傷記憶，從而逐步降低創傷事件帶來的恐懼反應，大大縮短療程。')
Context: (2675, '織夢技術可以應用於創傷後壓力症候群（PTSD）的治療，因為它能夠在一個絕對安全的夢境環境中，讓患者在治療師的引導下，以可控的方式反覆面對和處理他們的創傷記憶，從而逐步降低創傷事件帶來的恐懼反應，並大大縮短療程。')


In [47]:
query = "請總結織夢技術在「加速學習」和「專業培訓」這兩個領域的應用場景。"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (461, '織夢技術在「加速學習」領域的應用場景包括語言學習者在夢中與虛擬母語人士進行數小時的流利對話，從而強化語言迴路，以及醫學生在虛擬手術台上反覆練習複雜手術流程，直到形成肌肉記憶，將數週甚至數月的刻意練習濃縮在睡眠之中，從而指數級提升學習效率。')
Context: (2792, '織夢技術在「加速學習」領域的應用場景包括讓語言學習者在夢中與虛擬母語人士進行數小時的流利對話，從而讓大腦在睡眠中自動強化相關的語言迴路；同時，醫學生也能夠在虛擬手術台上反覆練習複雜的手術流程，直至形成肌肉記憶，這相當於將數週甚至數月的刻意練習濃縮在幾晚的睡眠之中，從而實現學習效率的指數級提升。在「專業培訓」領域，航空公司可以利用此技術模擬各種極端的飛行故障，讓飛行員在高壓環境下進行決策訓練；消防部門則可以演練火場救援，這些訓練不僅包含視覺模擬，還能模擬壓力、高溫和體力消耗等生理感受，其效果遠超傳統的模擬器，並能在零風險的條件下完成。')


In [48]:
query = "「神經織網」系統相較於早期的「Lucid」計畫，最關鍵的技術進步是什麼？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (863, '與早期的「Lucid」計畫相比，「神經織網」系統最關鍵的技術進步在於它不僅能夠「讀取」夢境線索，還能透過精準的聚焦超音波脈衝對其進行「寫入」操作，從而逆向活化特定的神經通路，實現對夢境內容的建構，這使得研究人員首次擁有了建構夢境內容的能力。')
Context: (2732, '相較於早期的「Lucid」計畫，關鍵的技術進步是「神經織網」系統能夠從內部進行精密的「介接」，透過繪製與特定記憶、情感和感官體驗相關的神經通路圖譜，將代表概念的「神經纖維」編織成一段完整的、可預設的夢境體驗，進而實現對夢境內容的干預和塑造，而「Lucid」計畫僅能誘發「清醒夢」，讓做夢者意識到自己在做夢，卻無法對夢境內容進行實質性干預。')


In [49]:
query = "為什麼《墨菲斯法案》的頒布是必要的？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (733, '《墨菲斯法案》的頒布是必要的，因為它嚴格限制了織夢技術的商業應用範圍，禁止其用於廣告和司法審訊，並確立了「思想與記憶的絕對隱私權」，以應對腦機介面與意識干預技術所帶來的潛在挑戰，例如侵犯個人最深層的恐懼、慾望和秘密，以及記憶篡改與偽造等倫理困境。')
Context: (2650, '《墨菲斯法案》（The "Morpheus" Act）的頒布是必要的，因為它旨在應對織夢技術帶來的倫理困境，例如記憶篡改與偽造、夢境隱私以及潛在的心理依賴等問題。')


In [50]:
query = "文件中是否提到了織夢技術在軍事領域的任何應用？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (488, '文件中沒有提到織夢技術在軍事領域的任何應用。')
Context: (2611, '根據提供的資料，織夢技術在軍事領域的應用並未被提及。')


In [51]:
query = "Aris Thorne 博士是否直接參與了「神經織網」的發明？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (776, 'Aris Thorne 博士奠定了織夢技術的基礎，但「神經織網」系統是由他的學生 Evelyn Reed 博士於 2011 年發明的。')
Context: (2653, 'Aris Thorne 博士並沒有直接參與「神經織網」的發明，而是由他的學生 Evelyn Reed 博士基於 Thorne 博士早期的清醒夢研究，在 21 世紀初發明了「神經織網」系統。')
